In [1]:
from ipynb.fs.full.feature_engineering_final import X_Transformer
from ipynb.fs.full.feature_engineering_final import TargetNormalizedRegressor
import pandas as pd
from ipynb.fs.full.feature_engineering_final import DataLoader
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import make_scorer
import numpy as np
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_predict
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
import seaborn as sns
from sklearn.feature_selection import RFECV
from sklearn.metrics import mean_squared_error
import numpy.ma as ma
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from scipy.stats import skew
from scipy.stats import boxcox_normmax
from scipy.special import boxcox1p
import seaborn as sns

In [2]:
def rmsle(y_true, y_pred, sample_weight=None, multioutput='uniform_average'):
    return np.sqrt(mean_squared_log_error(y_true, y_pred, sample_weight, multioutput))
    
#I just want to make rmse positive
flipped_score = make_scorer(rmsle, greater_is_better=True)

In [3]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5),scoring=None):
   
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes,scoring = scoring)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [4]:
data_loader = DataLoader("train.csv","test.csv")

/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3429: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


In [5]:
X_train = data_loader.getX_train()
Y_train = data_loader.getY_train()

In [6]:
x_transformer = X_Transformer()

In [7]:
X_train = x_transformer.fit_transform(X_train)
kf = KFold(n_splits=10,shuffle = True,random_state = 27)

In [8]:
#X_train = pd.DataFrame(X_train)
#overfit = []
#for i in X_train.columns:
#    counts = X_train[i].value_counts()
#    zeros = counts.iloc[0]
#    if zeros / len(X_train) * 100 > 99.94:
#        overfit.append(i)

#overfit = list(overfit)
#X_train = X_train.drop(overfit, axis=1)

In [9]:
#X_train = pd.DataFrame(X_train)
#numeric_dtypes = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#skew_features = X_train.apply(lambda x: skew(x)).sort_values(ascending=False)
#skew_features
#high_skew = skew_features[skew_features > 0]
#high_skew
#skew_index = high_skew.index
#for i in skew_index:
#    X_train[i] = boxcox1p(X_train[i], boxcox_normmax(X_train[i] + 1)) 
#np.sort(skew_index)


In [10]:
#print("Optimal number of features : %d" % selector.n_features_)

#plt.figure()
#plt.xlabel("Number of features selected")
#plt.ylabel("Cross validation score (nb of correct classifications)")
#plt.plot(range(1, (len(selector.grid_scores_))*10 ,10), selector.grid_scores_)
#plt.show()

In [11]:
#feature_selection_mask = np.loadtxt("features_mask.csv",delimiter = ',')
#filtered_indexes = list(map(lambda el2: el2[0],filter(lambda el: el[1]==0, enumerate(feature_selection_mask))))
#X_train = np.delete(X_train,filtered_indexes,axis = 1)


In [12]:
#cross validation and predicted vs measured cross validation
xgb_regressor_cross_val = TargetNormalizedRegressor(XGBRegressor(objective='reg:squarederror',eval_metric = 'rmse',learning_rate=0.01,n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006))
scores = cross_val_score(xgb_regressor_cross_val, X_train, Y_train, cv=kf,n_jobs=-1,scoring = flipped_score)
print("Rmse log average: %0.5f (+/- %0.5f)" % (scores.mean(), scores.std() * 2))
print("Rmse log median: %0.5f" % (np.median(scores)))
predicted = cross_val_predict(xgb_regressor_cross_val, X_train, Y_train, cv=kf,n_jobs=-1)
print(scores)
fig, ax = plt.subplots()
ax.scatter(Y_train, predicted, edgecolors=(0, 0, 0))
ax.plot([Y_train.min(), Y_train.max()], [Y_train.min(), Y_train.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

TypeError: sequence item 0: expected str instance, numpy.int64 found

In [ ]:
#learning curve of best model
plot_learning_curve(xgb_regressor_cross_val, "Title", X_train, Y_train, None, cv=kf, n_jobs=-1,scoring = flipped_score)

In [ ]:
tuned_parameters = {
        'max_depth': [5],
        'eta': [0.005],
        'n_estimators' : [200],
        #'gamma' : [0,0.1,0.5,1,5,10],
        #'min_child_weight': [1,2,5,10,30,50],
        #'max_delta_step' : [,10,100,1000,10000,100000,1000000000],
        'subsample' : [0.5],
        #'colsample_bytree' : [0.5,0.8,1],
        #'colsample_bylevel' : [0.5,0.8,1],
        #'colsample_bynode' : [0.5,0.8,1],
        #'lambda' : [0.1,100],
        #'alpha' : [0.1,100]
        
        }
clf = GridSearchCV(XGBRegressor(objective='reg:squarederror',eval_metric = 'rmsle'), tuned_parameters, cv=kf,scoring='neg_mean_squared_log_error',n_jobs=-1)
clf.fit(X_train, Y_train)
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.5f (+/-%0.05f) for %r" % (mean, std * 2, params))tuned_parameters = {
        'max_depth': [5],
        'eta': [0.005],
        'n_estimators' : [200],
        #'gamma' : [0,0.1,0.5,1,5,10],
        #'min_child_weight': [1,2,5,10,30,50],
        #'max_delta_step' : [,10,100,1000,10000,100000,1000000000],
        'subsample' : [0.5],
        #'colsample_bytree' : [0.5,0.8,1],
        #'colsample_bylevel' : [0.5,0.8,1],
        #'colsample_bynode' : [0.5,0.8,1],
        #'lambda' : [0.1,100],
        #'alpha' : [0.1,100]
        
        }
clf = GridSearchCV(XGBRegressor(objective='reg:squarederror',eval_metric = 'rmsle'), tuned_parameters, cv=kf,scoring='neg_mean_squared_log_error',n_jobs=-1)
clf.fit(X_train, Y_train)
print("Best parameters set found on development set:")
print()
print(clf.best_params_)
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.5f (+/-%0.05f) for %r" % (mean, std * 2, params))

In [ ]:
#regressor = XGBRegressor(objective='reg:squarederror',eval_metric = 'rmsle',max_depth = 5,eta = 0.005,n_estimators = 200,subsample = 0.5)
#regressor.fit(X_train, Y_train)
#test_data = pd.read_csv('test.csv')
#X_test = test_data.drop(['Id'], axis = 1)
#X_test = x_transformer.transform(X_test)
#X_test = np.delete(X_test,filtered_indexes,axis = 1)
#Ypredicted = regressor.predict(X_test)
#resultDf = pd.DataFrame()
#resultDf['Id'] = test_data['Id']
#resultDf['SalePrice'] = Ypredicted
#resultDf
#resultDf.to_csv('submission3.csv',index = False)

In [ ]:
#x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.1, random_state=30)
# fit model no training data
#xgb_regressor_early_stopping = XGBRegressor(objective='reg:squarederror',eval_metric = 'rmse',max_depth = 3,eta = 0.005,n_estimators = 3500,subsample = 0.5)
#eval_set = [(x_test, y_test)]
#xgb_regressor_early_stopping.fit(x_train, y_train, eval_metric="rmse", eval_set=eval_set,early_stopping_rounds = 3000)
# make predictions for test data
#y_pred = xgb_regressor_early_stopping.predict(x_test)
# evaluate predictions
#err = rmsle(y_test, y_pred)
#print("rmsle: %.5f%%" % (err))
#print("n_tree: %s" % (xgb_regressor_early_stopping.best_ntree_limit))